In [ ]:
!git clone https://github.com/matterport/Mask_RCNN.git

In [ ]:
%cd /content/Mask_RCNN

tensorflow==1.15
keras==2.2.5
h5py==2.10.0
scikit-image==0.16.2

In [ ]:
!pip install -r ./requirements.txt

In [4]:
import os
import sys
import json
import numpy as np
import skimage.draw
import cv2
from mrcnn.config import Config
from mrcnn import utils
from mrcnn import model as modellib
import matplotlib.pyplot as plt

Using TensorFlow backend.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!unzip /content/drive/MyDrive/Project/train.zip -d /content/drive/MyDrive/Project/dataset

In [ ]:
!unzip /content/drive/MyDrive/Project/val.zip -d /content/drive/MyDrive/Project/dataset

In [6]:
ROOT_DIR = "/content/drive/MyDrive/Project"
sys.path.append(ROOT_DIR)

COCO_WEIGHTS_PATH = os.path.join(ROOT_DIR, "mask_rcnn_coco.h5")
DEFAULT_LOGS_DIR = os.path.join(ROOT_DIR, "logs")

In [ ]:
class CustomConfig(Config):
    """Configuration for training on the custom  dataset.
    Derives from the base Config class and overrides some values.
    """
    # Give the configuration a recognizable name
    NAME = "object"


    IMAGES_PER_GPU = 2

    # Number of classes (including background)
    NUM_CLASSES = 1 + 4  # Background + pudding, milk, juice, pasta

    # Number of training steps per epoch
    STEPS_PER_EPOCH = 145

    # Skip detections with < 90% confidence
    DETECTION_MIN_CONFIDENCE = 0.9

config = CustomConfig()
config.display()

In [8]:
class CustomDataset(utils.Dataset):

    def load_dataset(self, dataset_dir, subset):
      
        # Add classes.
        self.add_class("object", 1, "Pudding")
        self.add_class("object", 2, "Milk")
        self.add_class("object", 3, "Pasta")
        self.add_class("object", 4, "Juice")


     
        # Train or validation dataset?
        assert subset in ["train", "val"]
        dataset_dir = os.path.join(dataset_dir, subset)
        
      
        annotations = json.load(open(os.path.join(dataset_dir, 'annotations.json')))
        #keep the name of the json files in the both train and val folders
        
        print("annotations", annotations)
        annotations = list(annotations.values())  # don't need the dict keys



        # The VIA tool saves images in the JSON even if they don't have any
        # annotations. Skip unannotated images.
        annotations = [i for i in annotations if i['regions']]
        
        # Add images
        for img in annotations:
            print("img", img)
            # Get the x, y coordinaets of points of the polygons.
            # There are stores in the shape_attributes (see json format above)
            polygons = [i['shape_attributes'] for i in img['regions']] 
            objects_names = [i['region_attributes']['names'] for i in img['regions']]
            
            print("objects names:",objects_names)
            name_dict = {"Pudding": 1,"Milk": 2, "Pasta" : 3, "Juice" : 4}

            # key = tuple(name_dict)
            num_ids = [name_dict[i] for i in objects_names]
            print("num_ids", num_ids)

            ## ADD_İMAGE FONKSİYONUNA PARAMETRELERİ GÖNDEREBİLMEK İÇİN DÜZENLEMELİYİZ
            image_path = os.path.join(dataset_dir, img['filename'])
            image = skimage.io.imread(image_path)
            height, width = image.shape[:2] # returns height, width, channel

            self.add_image(
                "object",
                image_id=img['filename'],  # use file name as a unique image id
                path=image_path,
                width=width, height=height,
                polygons=polygons,
                num_ids=num_ids
                )

    def load_mask(self, image_id):

        image_info = self.image_info[image_id]
        if image_info["source"] != "object": # check type of the images
            return super(self.__class__, self).load_mask(image_id)

        # Convert polygons to a bitmap mask of shape
        # [height, width, instance_count]
        info = self.image_info[image_id]
        if info["source"] != "object":
            return super(self.__class__, self).load_mask(image_id)

        num_ids = info['num_ids']
        mask = np.zeros([info["height"], info["width"], len(info["polygons"])],
                        dtype=np.uint8)
        
        for i, p in enumerate(info["polygons"]):
            # Get indexes of pixels inside the polygon and set them to 1
          rr, cc = skimage.draw.polygon(p['all_points_y'], p['all_points_x'])

          mask[rr, cc, i] = 1

        # Return mask, and array of class IDs of each instance. Since we have
        # one class ID only, we return an array of 1s
        # Map class names to class IDs.
        num_ids = np.array(num_ids, dtype=np.int32)
        return mask, num_ids #np.ones([mask.shape[-1]], dtype=np.int32)

    def image_reference(self, image_id):
        """Return the path of the image."""
        info = self.image_info[image_id]
        if info["source"] == "object":
            return info["path"]
        else:
            super(self.__class__, self).image_reference(image_id)
			


In [9]:
def train(model):
    # Training dataset.
    dataset_train = CustomDataset()
    dataset_train.load_dataset("/content/drive/MyDrive/Project/dataset", "train")
    dataset_train.prepare()

    # Validation dataset
    dataset_val = CustomDataset()
    dataset_val.load_dataset("/content/drive/MyDrive/Project/dataset", "val")
    dataset_val.prepare()


    print("Training network heads")
    model.train(dataset_train, dataset_val,
                learning_rate=config.LEARNING_RATE,
                epochs=500,
                layers='heads')

In [ ]:
config = CustomConfig()
model = modellib.MaskRCNN(mode="training", config=config,
                                  model_dir=DEFAULT_LOGS_DIR)

weights_path = COCO_WEIGHTS_PATH
        # Download weights file
if not os.path.exists(weights_path):
  utils.download_trained_weights(weights_path)

model.load_weights(weights_path, by_name=True, exclude=[
            "mrcnn_class_logits", "mrcnn_bbox_fc",
            "mrcnn_bbox", "mrcnn_mask"])

train(model)	

In [ ]:
# we define a prediction configuration 
class PredictionConfig(Config):
    NAME = "object"
    NUM_CLASSES = 1 + 4
    DETECTION_MIN_CONFIDENCE = 0.90
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1


# evaluate_model is used to calculate mean Average Precision of the model
def evaluate_model(dataset, model, cfg):
    APs = list()
    for image_id in dataset.image_ids:

        image, image_meta, gt_class_id, gt_bbox, gt_mask = modellib.load_image_gt(dataset, cfg, image_id, use_mini_mask=False)

        scaled_image = modellib.mold_image(image, cfg)

        sample = np.expand_dims(scaled_image, 0)

        yhat = model.detect(sample, verbose=0)

        r = yhat[0]

        AP, _, _, _ = utils.compute_ap(gt_bbox, gt_class_id, gt_mask, r["rois"], r["class_ids"], r["scores"], r['masks'])

        APs.append(AP)
    mAP = np.mean(APs)
    return mAP

# train dataset
train_set = CustomDataset()
train_set.load_dataset("/content/drive/MyDrive/Project/dataset", "train")
train_set.prepare()

# Validation dataset
val_set = CustomDataset()
val_set.load_dataset("/content/drive/MyDrive/Project/dataset", "val")
val_set.prepare()

# create config
cfg = PredictionConfig()
# define the model
model = modellib.MaskRCNN(mode='inference', model_dir='./', config=cfg)
# load model weights
model.load_weights('/content/drive/MyDrive/Project/logs/object20220422T0302/mask_rcnn_object_0080.h5', by_name=True)



In [11]:
# evaluate model on train dataset
train_mAP = evaluate_model(train_set, model, cfg)
print(train_mAP)

0.9835958005249343


In [12]:
# evaluate model on test dataset
val_mAP = evaluate_model(val_set, model, cfg)
print(val_mAP)

0.925
